In [ ]:
from pathlib import Path
import json
import re

import requests
import geopandas as gpd
import pandas as pd
import pycountry

In [ ]:
GEOJSON_URL = "https://github.com/NREL/EnergyPlus/raw/develop/weather/master.geojson"

In [ ]:
r = requests.get(GEOJSON_URL)
r.raise_for_status()

data = r.json()

In [ ]:
PROCESS_LINK_KEYS = ['epw', 'ddy', 'stat', 'all', 'dir']

In [ ]:
for feature in data["features"]:
    for k in PROCESS_LINK_KEYS:
        if k in feature["properties"]:
            feature["properties"][k] = feature["properties"][k].split("href=")[1].split(">")[0]

In [ ]:
with open('eplus_processed_master.geojson', 'w') as f:
    json.dump(data, fp=f, indent=4)

In [ ]:
df_geo = gpd.read_file('eplus_processed_master.geojson')
df_geo.shape

In [ ]:
df_geo.crs

In [ ]:
df_geo.columns

In [ ]:
type(data["features"])

In [ ]:
feature = data["features"][0]
feature.keys()

In [ ]:
feature['type']

In [ ]:
feature["geometry"]

In [ ]:
feature["properties"].keys()

In [ ]:
feature

In [ ]:
df_geo = gpd.read_file(GEOJSON_URL)

In [ ]:
def process_link(link):
    return link.split("href=")[1].split(">")[0]

In [ ]:
for k in PROCESS_LINK_KEYS:
    df_geo[k] = df_geo[k].apply(process_link)

In [ ]:
print("\n".join(df_geo['title'].values[:10]))

In [ ]:
title = 'DZA_Algiers.603900_IWEC'

In [ ]:
RE_TITLE = re.compile(r'(?P<country_code>[a-zA-Z]{3})_(?P<city>.*)\.(?P<wmo>\d{6})_(?P<type>\w+)')

In [ ]:
for title in df_geo['title']:
    if (m := RE_TITLE.match(title)):
        pass
    else:
        print(title)

In [ ]:
df_geo[['country_code', 'city', 'wmo', 'type']] = df_geo['title'].str.extract(RE_TITLE)

In [ ]:
pycountry.countries.get(alpha_3='DZA')

In [ ]:
usa = pycountry.countries.get(alpha_3='USA')

In [ ]:
usa.flag

In [ ]:
pycountry.countries.lookup('allemagne')

In [ ]:
pycountry.countries.search_fuzzy('Germ')

In [ ]:
df_geo.plot()

# Dump pycountry database to a usable one?

In [ ]:
list(pycountry.countries)

In [ ]:
# Can just download this one:
!wget https://raw.githubusercontent.com/flyingcircusio/pycountry/main/src/pycountry/databases/iso3166-1.json

# climate.onebuilding

In [ ]:
from lxml import etree, html

In [ ]:
KML_DIR = Path('kmls')

kml_links = [x for x in """
https://climate.onebuilding.org/WMO_Region_1_Africa/Region1_Africa_EPW_Processing_locations.kml
https://climate.onebuilding.org/WMO_Region_2_Asia/Region2_Asia_EPW_Processing_locations.kml
https://climate.onebuilding.org/WMO_Region_2_Asia/Region2_Region6_Russia_EPW_Processing_locations.kml
https://climate.onebuilding.org/WMO_Region_3_South_America/Region3_South_America_EPW_Processing_locations.kml

https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/Region4_Canada_EPW_Processing_locations.kml
https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/Region4_USA_EPW_Processing_locations.kml
https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/Region4_NA_CA_Caribbean_EPW_Processing_locations.kml

https://climate.onebuilding.org/WMO_Region_5_Southwest_Pacific/Region5_Southwest_Pacific_EPW_Processing_locations.kml
https://climate.onebuilding.org/WMO_Region_6_Europe/Region6_Europe_EPW_Processing_locations.kml
https://climate.onebuilding.org/WMO_Region_7_Antarctica/Region7_Antarctica_EPW_Processing_locations.kml
""".split() if x.strip()]

In [ ]:
import urllib.request 

for kml_link in kml_links:
    outpath = KML_DIR / kml_link.split('/')[-1]
    if not outpath.is_file():
        print(f"Downloading {kml_link}")
        urllib.request.urlretrieve(kml_link, outpath)

In [ ]:
# !npm install -g @mapbox/togeojson
# !togeojson Region6_Europe_EPW_Processing_locations.kml > Region6_Europe_EPW_Processing_locations.geojson

In [ ]:
RE_NCEI_ISD_ERA = re.compile(r'(?P<source_type>.*) - #years=\[(?P<source_years>\d+)\] Period of Record=(?P<source_period>.*)')

def parse_description_text(description_text: str):
    # table = etree.HTML(description_text).find("body/table")
    table = html.fromstring(description_text)
    info = {}
    assert table is not None
    if len(table) not in [11, 12, 13]:
        raise ValueError(f"len is {len(table)}")
    for i, row in enumerate(table):
        tds = row.findall('td')
        assert len(tds) == 1
        #print(etree.tostring(row, pretty_print=True).decode())
        td = tds[0]
        if i == 0:
            assert(td.attrib['colspan'] == '2')
            info['epw_name'] = td.text_content()
            continue
        # b = td.find('b')
        # if b is not None:
        #     td = b
        text = td.text_content().strip()
        if text.startswith('Data Source'):
            info['Data Source'] = text[12:]
            continue
        if text.startswith('ERA5') or text.startswith('NCEI') or text.startswith('NSRDB'):
            m = RE_NCEI_ISD_ERA.match(text)
            assert m is not None, text
            info.update(RE_NCEI_ISD_ERA.match(text).groupdict())
            continue
        if text.startswith('WMO'):
            info['WMO'] = text[4:]
            continue
        if '°' in text:
            splitted = [x.strip() for x in text.split('  ')]
            assert len(splitted) == 2, f"'{text}'"
            lat_degrees, lon_degrees = splitted
            info['latitude_degrees'] = lat_degrees
            info['longitude_degrees'] = lon_degrees
            continue
        if text.startswith('Elevation'):
            info['Elevation'] = text[10:]
            continue
        if text.startswith('Time Zone'):
            info['Time Zone'] = text.split('{')[1].split('}')[0]
            continue
        if text.startswith('Design conditions'):
            info['Design conditions'] = text[18:]
            continue
        if text.startswith('99% Heating DB'):
            info['99% Heating DB'] = text[15:]
            continue
        if text.startswith('1% Cooling DB'):
            info['1% Cooling DB'] = text[14:]
            continue
        if text.startswith('HDD'):
            hdd, cdd = [x.strip() for x in 'HDD18 3705, CDD10 711'.split(',')]
            khdd, vhdd = hdd.split(' ')
            assert khdd == 'HDD18'
            info[khdd] = float(vhdd)
            kcdd, vcdd = cdd.split(' ')
            assert kcdd == 'CDD10'
            info[kcdd] = float(vcdd)
            continue
        if text.startswith('URL'):
            info['URL'] = text[4:]
            continue
        if text.startswith('ASHRAE HOF 2021 Climate Zone'):
            info['Climate Zone Standard'] = "ASHRAE HOF 2021 "
            info['Climate Zone'] = text[29:]
            continue
        if text.startswith('ASHRAE HOF 2017 Climate Zone'):
            info['Climate Zone Standard'] = "ASHRAE HOF 2017 "
            info['Climate Zone'] = text[29:]
            continue
        assert False, text
        
    return info

In [ ]:
RE_KML_NAME = re.compile(r'Region(?P<RegionNumber>\d)_(?P<RegionName>.*)_EPW_Processing_locations.kml')
def process_kml_file(kml_path: Path):
    tree = etree.parse(kml_path)
    root = tree.getroot()
    # document = root.getchildren()[0]
    ns = root.nsmap[None]
    namespaces={'k': ns}

    region_m = RE_KML_NAME.match(kml_path.name)
    assert region_m is not None
    region_dict = region_m.groupdict()
    
    all_infos = []
    for placemark in root.xpath(f'.//k:Placemark', namespaces=namespaces):
        #name = placemark.find('k:name', namespaces=namespaces)
        name = placemark.find('k:name', namespaces=namespaces)
        assert name is not None
        name = name.text
        desc = placemark.find('k:description', namespaces=namespaces)
        if desc is None:
            print(f"Skipping no description: text='{placemark.text.strip()}' {name=}")
            continue

        styleUrl = placemark.find('k:styleUrl', namespaces=namespaces)
        point = placemark.find('k:Point', namespaces=namespaces)
        assert styleUrl.text == "#weatherlocation"
        assert point is not None
        altitudeMode = point.find('k:altitudeMode', namespaces=namespaces)
        assert altitudeMode.text == 'absolute'
        coordinates = point.find('k:coordinates', namespaces=namespaces)
        coords = coordinates.text.split(',')
        assert len(coords) == 3
        (longitude, latitude, altitude) = [float(x) for x in coords]
        try:
            parse_description_text(desc.text)
        except:
            raise ValueError(f"Skipping not 12: text='{placemark.text.strip()}' {name=}")

        d = dict(
            {
                'name': name,
                'latitude': latitude,
                'longitude': longitude,
                'altitude': altitude
            },
            **parse_description_text(description_text=desc.text),
            **region_dict,
        )
        all_infos.append(d)

    return all_infos

In [ ]:
kml_paths = sorted(list(KML_DIR.glob('*.kml')))
kml_paths

In [ ]:
all_infos = []
for kml_path in kml_paths:
    print(f"Processing {kml_path}")
    all_infos += process_kml_file(kml_path=kml_path)
    print(f"\n\nDone Processing {kml_path}\n")

In [ ]:
len(all_infos)

In [ ]:
df = pd.DataFrame(all_infos)

In [ ]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(x=df['longitude'], y=df['latitude'], z=df['altitude']), crs="EPSG:4326"
)

In [ ]:
gdf.to_file('onebuilding.geojson')

In [ ]:
with open('onebuilding.geojson', 'r') as f:
    data = json.load(f)
with open('onebuilding_pretty.geojson', 'w') as f:
    json.dump(data, fp=f, indent=4)

In [ ]:
df_geo.columns

In [ ]:
gdf[['name',
 'epw_name',
 'geometry']].to_file('onebuilding_short.geojson')

# Find closest weater stations

In [ ]:
p = Path("~/Software/Others/OS-build-release/Products/python").expanduser()
import sys
sys.path.insert(0, str(p))
import openstudio

In [ ]:
# Lorient, France
latitude, longitude = (47.73665335117919, -3.368750001289318)

## E+ database

In [ ]:
distances = df_geo['geometry'].apply(lambda p: openstudio.getDistanceLatLon(lat1=latitude, lon1=longitude, lat2=p.y, lon2=p.x)) / 1000.0
distances.name = 'distance_km'

In [ ]:
distances.min()

In [ ]:
df_geo.merge(distances.sort_values(ascending=True).iloc[:5], left_index=True, right_index=True).sort_values(by='distance_km', ascending=True)

## Onebuilding

In [ ]:
distances = gdf['geometry'].apply(lambda p: openstudio.getDistanceLatLon(lat1=latitude, lon1=longitude, lat2=p.y, lon2=p.x)) / 1000.0
distances.name = 'distance_km'
gdf.merge(distances.sort_values(ascending=True).iloc[:5], 
             left_index=True, right_index=True).sort_values(by='distance_km', ascending=True)